# Utility functions

> This module defines several utility functions that are used throughout findmycells

In [ ]:
#| default_exp utils

In [ ]:
#| export

from typing import List, Optional
from pathlib import Path, PosixPath

import numpy as np
from skimage import io
from skimage import measure
from shapely.geometry import Polygon
from shapely.validation import make_valid


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

def list_dir_no_hidden(path: PosixPath, only_dirs: Optional[bool]=False, only_files: Optional[bool]=False) -> List[PosixPath]:
    if only_dirs == True:
        detected_paths = [elem for elem in path.iterdir() if (elem.is_dir() == True) & (elem.name.startswith('.') == False)]
    elif only_files == True:
        detected_paths = [elem for elem in path.iterdir() if (elem.is_dir() == False) & (elem.name.startswith('.') == False)]
    else:
        detected_paths = [elem for elem in path.iterdir() if elem.name.startswith('.') == False]
    return detected_paths

In [ ]:
#| export

def load_zstack_as_array_from_single_planes(path: PosixPath, file_id: str, 
                                            minx: Optional[int]=None, maxx: Optional[int]=None, 
                                            miny: Optional[int]=None, maxy: Optional[int]=None) -> np.ndarray:
    types = list(set([type(minx), type(maxx), type(miny), type(maxy)]))    
    if any([minx, maxx, miny, maxy]):
        if (len(types) == 1) & (types[0] == int):
            cropping = True
        else:
            raise TypeError("'minx', 'maxx', 'miny', and 'maxy' all have to be integers - or None if no cropping has to be done")
    else:
        cropping = False
    matching_filepaths = [filepath for filepath in list_dir_no_hidden(path) if filepath.name.startswith(file_id)]
    cropped_zstack = []
    for single_plane_filepath in matching_filepaths:
        tmp_image = io.imread(single_plane_filepath)
        if cropping == True:
            tmp_image = tmp_image[minx:maxx, miny:maxy]
        cropped_zstack.append(tmp_image.copy())
        del tmp_image
    return np.asarray(cropped_zstack) 

In [ ]:
#| export

def unpad_x_y_dims_in_3d_array(padded_3d_array: np.ndarray, pad_width: int) -> np.ndarray:
    return padded_3d_array[:, pad_width:padded_3d_array.shape[1]-pad_width, pad_width:padded_3d_array.shape[2]-pad_width]

In [ ]:
#| export

def get_polygon_from_instance_segmentation(single_plane: np.ndarray, label_id: int) -> Polygon:
    x_dim, y_dim = single_plane.shape
    tmp_array = np.zeros((x_dim, y_dim), dtype='uint8')
    tmp_array[np.where(single_plane == label_id)] = 1
    tmp_contours = measure.find_contours(tmp_array, level = 0)[0]
    roi = Polygon(tmp_contours)
    if roi.is_valid == False:
        roi = make_valid(roi)
    return roi

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

SyntaxError: EOL while scanning string literal (<unknown>, line 40)